✅ Phase 2：工具層與資料層 — 完整待辦清單（通用 Agent 版）
2.1 Data Ingestion & Dataset Profiling（必做）
2.1.1 資料讀取（CSV / Parquet）

🎯 目的
讓 agent 能讀任何 tabular dataset，而不是寫死某一份資料。

📦 輸出物

load_dataset(path) -> DataFrame

支援：

CSV（必做）

Parquet（可選）

✅ Done

換任意 CSV path 都能正常讀

讀不到檔案 / 副檔名不支援 → 友善錯誤

🧠 通用性重點

不假設 delimiter / encoding 永遠正確

不在這一步做任何分析

2.1.2 Dataset Profile（Schema Summary）

🎯 目的
讓 Planner / LLM 在「不看 raw data」的情況下理解資料長怎樣。

📦 輸出物（JSON-friendly）

dataset level

n_rows, n_cols, file_type

column level（每一欄）

name

raw dtype

missing_rate

n_unique

sample_values（<=5）

is_numeric_candidate

is_categorical_candidate

is_id_like

numeric_parseable（object → numeric）

✅ Done

輸出可 json.dumps

對任何 CSV 都能產生完整摘要

🧠 通用性重點

不相信 pandas dtype

用 cardinality + ratio 判斷欄位性質

2.1.3 欄位解釋檔（Data Dictionary）Normalize（你特別提到的）

🎯 目的
讓 agent 能利用 domain knowledge（而不是只看欄位名）。

📦 輸出物

將 CSV / Excel / Markdown 的欄位說明 → 統一成：

{
  "columns": [
    {"name": "...", "description": "...", "type_hint": "..."}
  ]
}


✅ Done

dictionary 缺欄位 / 多欄位 → 不會炸

欄位名大小寫 / snake/camel 能對齊

🧠 通用性重點

dictionary 是 optional

永遠不能假設一定存在

2.2 Target Variable Inference（核心）
2.2.1 Deterministic Target Heuristic（必做，fallback）

🎯 目的
確保 agent 在「沒有 LLM / dictionary」時仍然能跑。

📦 輸出物

[
  {
    "column": "quality",
    "score": 42.0,
    "reasons": ["keyword_match:quality", "numeric_moderate_unique"]
  }
]


✅ Done

永遠至少回 1 個候選

排序穩定、可解釋

🧠 通用性重點

不 hardcode dataset 名稱

heuristic ≠ 最終真相，只是候選產生器

2.2.2 Gemini-based Semantic Target Ranking（可選但強烈建議）

🎯 目的
利用欄位解釋 + question，做語意對齊（通用性大幅提升）。

📦 輸出物（嚴格 JSON）

{
  "target_ranking": [
    {"column": "quality", "confidence": 0.92, "reasons": ["matches question"]}
  ],
  "task_type": "regression"
}


✅ Done

JSON parse 失敗 → fallback heuristic

confidence < threshold → 不覆蓋 heuristic

🧠 通用性重點

LLM 是 advisor，不是 dictator

成本可控（只 rerank top-K）

2.3 Task Type Inference（classification vs regression）

🎯 目的
讓 Planner 知道後面該用哪一類工具。

📦 輸出物

{"task_type": "regression", "reason": "..."}


✅ Done

numeric / categorical 邏輯正確

ordinal target（如 rating）可先當 regression

🧠 通用性重點

規則清楚、可 override

2.4 Reusable Analysis Tools（通用工具層）
2.4.1 describe_df

🎯 目的
快速 human-style data understanding。

📦 輸出物

missing summary

numeric stats

categorical value counts（top-N）

2.4.2 correlation_to_target（你現在正在做的）

🎯 目的
找「最直接的訊號」。

📦 輸出物

Pearson + Spearman

direction + abs_corr

top-K features

2.4.3 train_baseline_model

🎯 目的
給出「能跑的 baseline」。

📦 輸出物

model type

metrics（RMSE/R² 或 Accuracy/F1）

coefficients / feature importance

2.5 Tool Orchestration（Planner 能用）

🎯 目的
讓 Planner 真的能「選工具」，而不是寫死流程。

📦 輸出物

[
  {"tool": "correlation_to_target", "args": {...}},
  {"tool": "train_baseline_model", "args": {...}}
]


✅ Done

Tool node 能依 plan 執行

tool error 不會炸整個 graph

2.6 Structured Analysis Output（Agentic 必備）

🎯 目的
讓結果可解釋、可追蹤、可重跑。

📦 輸出物

{
  "dataset_profile": {...},
  "target": "...",
  "task_type": "...",
  "correlations": {...},
  "baseline_model": {...},
  "warnings": [...]
}

2.7 Error Handling & Guardrails（不要跳過）

🎯 目的
避免 agent hallucinate 或 silent failure。

📦 輸出物

tool-level error payload

warnings（high missing, id leakage）